In [8]:
# --- Config (edit these if needed) ---
weights_path = "weights/best.pt"           # Custom trained YOLOv8 model weights
video_path   = "assets/CAR_vid.mp4"       # Place your video file in assets/ folder
conf_thresh  = 0.5                        # Detection confidence threshold

# Training is disabled by default to prevent accidental runs
RUN_TRAINING = False



In [ ]:
"""
Training
- I trained locally for 20 epochs to validate the pipeline.
- Final 100-epoch training was done on a GPU with ~3000 images.
- This cell is OFF by default to avoid accidental runs and dataset path issues.
"""
if RUN_TRAINING:
    from ultralytics import YOLO
    
    data_yaml = "data/data.yaml"  # path to your data.yaml file
    model = YOLO("yolov8n.pt")    # base model
    model.train(data=data_yaml, epochs=20)


In [ ]:
import cv2
import cvzone
import os
import csv
import datetime
from ultralytics import YOLO
from paddleocr import PaddleOCR
import pandas as pd
import time



# Path configuration for notebook environment
if os.path.basename(os.getcwd()) == 'notebooks':
    weights_path = "../" + weights_path
    video_path = "../" + video_path

# Setup and validation
print(f"📁 Current directory: {os.getcwd()}")
print(f"🎯 Model weights: {weights_path} ({'✅' if os.path.exists(weights_path) else '❌'})")
print(f"🎬 Video file: {video_path} ({'✅' if os.path.exists(video_path) else '❌'})")

# Create results directory
results_dir = "../results" if os.path.basename(os.getcwd()) == 'notebooks' else "results"
os.makedirs(results_dir, exist_ok=True)


csv_filename = os.path.join(results_dir, f"license_plates_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
csv_headers = ['timestamp', 'frame_number', 'plate_text', 'confidence', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2']

frame_count = 0
detected_plates = []

# Initialize OCR and YOLO
ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log=False)
model = YOLO(weights_path if os.path.exists(weights_path) else "yolov8n.pt")
classnames = ['license-plate', 'vehicle']


cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise FileNotFoundError(f"❌ Cannot open video: {video_path}")

fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_delay = max(1, int(1000 / fps))

def clean_plate_text(text):
    if not text or len(text) < 2:
        return None
    cleaned = ''.join(c.upper() for c in text if c.isalnum() or c.isspace())
    cleaned = ' '.join(cleaned.split())
    return cleaned if 2 <= len(cleaned) <= 15 else None

def save_to_csv(data, filename):
    df = pd.DataFrame(data, columns=csv_headers)
    df.to_csv(filename, index=False)
    print(f"💾 Saved {len(data)} detections to {filename}")

print("🚀 Starting ALPR detection... (press 't' to stop)")
start_time = time.time()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("📹 End of video")
            break

        frame_count += 1
        results = model(frame, verbose=False)

        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])

                if conf > 0.5 and cls < len(classnames):
                    class_name = classnames[cls]

                    if class_name == 'license-plate':
                        crop = frame[max(0,y1-5):y2+5, max(0,x1-5):x2+5]
                        if crop.size > 0:
                            ocr_result = ocr.ocr(crop, cls=True)
                            if ocr_result and ocr_result[0]:
                                raw_text = ' '.join([line[1][0] for line in ocr_result[0] if line[1][0].strip()])
                                cleaned_text = clean_plate_text(raw_text)
                                if cleaned_text:
                                    detected_plates.append([
                                        datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                                        frame_count, cleaned_text, round(conf, 3),
                                        x1, y1, x2, y2
                                    ])
                                    display_text = f"{cleaned_text} ({conf:.2f})"
                                else:
                                    display_text = f"Unreadable ({conf:.2f})"
                            else:
                                display_text = f"No text ({conf:.2f})"
                        else:
                            display_text = f"Invalid crop"

                        color = (0, 255, 0)

                    else:
                        color = (255, 0, 0)
                        display_text = f"Vehicle ({conf:.2f})"

                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cvzone.putTextRect(frame, display_text, [x1 + 5, y1 - 10], scale=0.7, thickness=1, colorR=color)

        cvzone.putTextRect(frame, f"Frame {frame_count}/{total_frames} | Plates: {len(detected_plates)}", 
                           [10, 30], scale=0.7, thickness=1, colorR=(255,255,255))

        cv2.imshow("ALPR", frame)
        key = cv2.waitKey(frame_delay) & 0xFF
        if key == ord('t'):
            print("🛑 Stopping...")
            break

except KeyboardInterrupt:
    print("\n🛑 Interrupted by user")

finally:
    if detected_plates:
        save_to_csv(detected_plates, csv_filename)
    cap.release()
    cv2.destroyAllWindows()
    print("✅ Done in %.1fs" % (time.time() - start_time))


📁 Current directory: d:\license-plate-recognition\notebooks
🎯 Model weights: ../weights/best.pt (✅)
🎬 Video file: ../assets/CAR_vid.mp4 (✅)
🚀 Starting ALPR detection... (press 't' to stop)
🛑 Stopping...
💾 Saved 34 detections to ../results\license_plates_20250911_160241.csv
✅ Done in 17.1s
